In [1]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 /(fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval= low, maxval= high, dtype=tf.float32)

In [ ]:
xavier_result = xavier_init(2, 3)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run(xavier_result)
#sess.close()

In [18]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus, optimizer = tf.train.AdamOptimizer(),scale = 0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(self.x + scale * tf.random_normal((n_input,)), self.weights['w1']), self.weights['b1']))
        
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
        
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        
        self.optimizer = optimizer.minimize(self.cost)
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
        #self.sess.run(self.reconstruction)
    
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype= tf.float32 ))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype = tf.float32)) 
        
        return all_weights
    
    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x: X, self.scale: self.training_scale})
    
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict = {self.x: X, self.scale: self.training_scale})
        return cost
                                        
    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X, self.scale: self.training_scale})
                                        
    def generate(self, hidden = None):
        if hidden is None:
            hidden = np.random.normal(size = self.weights["b1"])
        return self.sess.run(self.reconstruction, feed_dict = {self.hidden: hidden})
                                        
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x: X, self.scale: self.training_scale}) 
                                        
    def getWeights(self):
        return self.sess.run(self.weights['w1']) 
                                        
    def getBiases(self):
        return self.sess.run(self.weights['b1'])                                

In [4]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test

In [6]:
def get_random_block_from_data(data, batch_szie):
    start_index = np.random.randint(0, len(data) - batch_szie)
    return data[start_index:(start_index + batch_szie)]

In [7]:
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)

In [8]:
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

In [19]:
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784, n_hidden=200, 
                                               transfer_function= tf.nn.softplus,
                                              optimizer = tf.train.AdamOptimizer(learning_rate= 0.001),
                                              scale = 0.01)

In [21]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)
        
        cost = autoencoder.partial_fit(batch_xs)
        
        avg_cost += cost / n_samples * batch_size
    
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

Epoch: 0001 cost= 18843.489904545
Epoch: 0002 cost= 12831.190327273
Epoch: 0003 cost= 11268.817311364
Epoch: 0004 cost= 10624.070675000
Epoch: 0005 cost= 9960.618135795
Epoch: 0006 cost= 9145.861361364
Epoch: 0007 cost= 7996.029322159
Epoch: 0008 cost= 8474.344046591
Epoch: 0009 cost= 9218.925573864
Epoch: 0010 cost= 8026.405338636
Epoch: 0011 cost= 8945.935981250
Epoch: 0012 cost= 8719.871868750
Epoch: 0013 cost= 8882.326907955
Epoch: 0014 cost= 8777.940461364
Epoch: 0015 cost= 7766.417610227
Epoch: 0016 cost= 8396.626248295
Epoch: 0017 cost= 8109.933447727
Epoch: 0018 cost= 7938.962407955
Epoch: 0019 cost= 8368.181596591
Epoch: 0020 cost= 8144.320295455


In [22]:
print("Total cost:" + str(autoencoder.calc_total_cost(X_test)))

Total cost:680297.0


In [30]:
print("feature is", autoencoder.reconstruct(X_test))

feature is [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [25]:
print("transform is", autoencoder.transform(X_test))

transform is [[  1.69223487e+00   1.65631950e+00   1.23161817e+00 ...,   1.14924908e+00
    9.24486339e-01   8.40896726e-01]
 [  7.13625669e+00   1.81487656e+00   3.57809258e+00 ...,   2.76933700e-01
    1.98595560e+00   3.76152468e+00]
 [  9.29345727e-01   7.09575593e-01   1.42033088e+00 ...,   1.63140130e+00
    2.02848744e+00   8.65556240e-01]
 ..., 
 [  1.82241178e+00   1.37665021e+00   1.79494393e+00 ...,   1.87624729e+00
    1.53245115e+00   2.25584960e+00]
 [  3.15694332e+00   4.26359754e-03   1.90830445e+00 ...,   1.66335583e+00
    6.77396154e+00   5.82486331e-01]
 [  2.88439059e+00   5.54185092e-01   1.78009069e+00 ...,   4.20405533e-09
    2.74945879e+00   8.77207756e-01]]


In [27]:
print("transform is", autoencoder.generate())

TypeError: expected sequence object with len >= 0 or a single integer